# Deep Learning
### Gender Recognition by Voice and Speech Analysis

This database was created to identify a voice as male or female, based upon acoustic properties of the voice and speech. The dataset consists of 3,168 recorded voice samples, collected from male and female speakers. The voice samples are pre-processed by acoustic analysis in R using the seewave and tuneR packages, with an analyzed frequency range of 0hz-280hz (human vocal range).

### The Dataset
The following acoustic properties of each voice are measured and included within the CSV:

* `meanfreq`: mean frequency (in kHz)
* `sd`: standard deviation of frequency
* `median`: median frequency (in kHz)
* `Q25`: first quantile (in kHz)
* `Q75`: third quantile (in kHz)
* `IQR`: interquantile range (in kHz)
* `skew`: skewness (see note in specprop description)
* `kurt`: kurtosis (see note in specprop description)
* `sp.ent`: spectral entropy
* `sf`: spectral flatness
* `mode`: mode frequency
* `centroid`: frequency centroid (see specprop)
* `peakf`: peak frequency (frequency with highest energy)
* `meanfun`: average of fundamental frequency measured across acoustic signal
* `minfun`: minimum fundamental frequency measured across acoustic signal
* `maxfun`: maximum fundamental frequency measured across acoustic signal
* `meandom`: average of dominant frequency measured across acoustic signal
* `mindom`: minimum of dominant frequency measured across acoustic signal
* `maxdom`: maximum of dominant frequency measured across acoustic signal
* `dfrange`: range of dominant frequency measured across acoustic signal
* `modindx`: modulation index. Calculated as the accumulated absolute difference between adjacent measurements of fundamental frequencies divided by the frequency range
* `label`: male or female

### Import dependencies

In [ ]:
# Dependencies
import numpy as np
import pandas as pd

# Preprocessing libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

# Neural network libraries
from keras.models import Sequential
from keras.layers import Dense

### Read the provided data

In [ ]:
voice = pd.read_csv('../Resources/voice.csv')
voice.head()

## Data Pre-Processing

Pull the target variable's column into a separate variable so it is not included in the model training.

In [ ]:
X = voice.drop("label", axis=1)
y = voice["label"]
print(X.shape, y.shape)

### Split the data into training and testing sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

### Scale the input features

In [ ]:
# Fit the scaler using ONLY the training data
X_scaler = StandardScaler().fit(X_train)

# Transform the values of training and testing sets using the trained scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### One-hot encode the categorical target variables

**NOTE:** Remember that encoding categorical values represented as strings must be done in 2 steps (see below):

In [ ]:
# Step 1: Label-encode data set to convert the string representations of the categories to numeric
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded numeric labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

print(f"Original target values: {y_train[-5:].values}")
print(f"Label-encoded target values: {encoded_y_train[-5:]}")
print(f"One-hot-encoded target values: \n {y_train_categorical[-5:]}")

### Create a Deep Learning Model

Create this model with **2 hidden layers**. Each layer should have **100 nodes**.

In [ ]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=20))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [ ]:
model.summary()

### Compile and fit the model

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

### Quantify our Trained Model

In [ ]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

<hr>

# Saving a trained model

Once a model has been trained, we will oftentimes want to save it to deploy in a separate application. We can save a trained Keras model using the `HDF5` binary format with the extension `.h5`.

[See here for more information on saving/loading a model](https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model)

In [ ]:
model.save("voice_model_trained.h5")

### Loading a trained model

In [ ]:
from keras.models import load_model
voice_model = load_model("voice_model_trained.h5")

### Using a loaded model
Once a model has been loaded, you can treat it as any other model.

In [ ]:
## Evaluating the loaded model

model_loss, model_accuracy = voice_model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")